In [10]:
import keras
import numpy as np
import tensorflow as tf

from datetime import datetime
from keras.regularizers import l1, l2
from models import Unet_Model, CNN_Model

In [2]:
def import_data(path):
    x_train = np.load(path + "/x_train.npy")
    x_val = np.load(path + "/x_val.npy")
    x_test = np.load(path + "/x_test.npy")
    y_train = np.load(path + "/y_train.npy")
    y_val = np.load(path + "/y_val.npy")
    y_test = np.load(path + "/y_test.npy")
    z_train = np.load(path + "/z_train.npy")
    z_val = np.load(path + "/z_val.npy")
    z_test = np.load(path + "/z_test.npy")

    return x_train, x_val, x_test, y_train, y_val, y_test, z_train, z_val, z_test

In [3]:
# import dataset
x_train, x_val, x_test, y_train, y_val, y_test, z_train, z_val, z_test = import_data("data128")

# Tuned Unet

In [5]:
# Parameter Tuning

# Regularization
regularization = [None, "l1", "l2"]

# Dropout rate
drop_rate = [0.1, 0.2, 0.3, 0.4, 0.5]

# Upsampling
up_sample = [False, True]

# Batch Normalization
batch_norm = [True, False]

# Learning Rate
learning_rate = [0.0001, 0.001, 0.01, 0.1]

best_configuration = dict(
    val_loss=float("inf"), 
    reg=regularization[0], 
    dr=drop_rate[1], 
    us=up_sample[0], 
    bn=batch_norm[1], 
    lr=learning_rate[1], 
    epoch_num=30
)

In [5]:
with open("unet_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)

    for reg in regularization:
        UNet_model = Unet_Model(input_shape=(128,128,1), init_filter=16, drop_rate=best_configuration["dr"], up_sampling=best_configuration["us"], 
                                regularization=reg, batch_norm=best_configuration["bn"])
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        UNet_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        UNet_results = UNet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=best_configuration["epoch_num"], verbose=0)
        val_loss, _ = UNet_model.evaluate(x_val, y_val, verbose=0)
        print(f"val_loss={val_loss} (reg={reg})")
        print(f"val_loss={val_loss} (reg={reg})\n", file=f)

        if val_loss < best_configuration["val_loss"]:
            print("*** best_configuration updated")
            print("*** best_configuration updated\n", file=f)
            best_configuration["val_loss"] = val_loss
            best_configuration["loss"] = UNet_results.history["loss"]
            best_configuration["reg"] = reg

            UNet_model.save("UNet_model.h5")
    
    for dr in drop_rate:
        UNet_model = Unet_Model(input_shape=(128,128,1), init_filter=16, drop_rate=dr, up_sampling=best_configuration["us"], 
                                regularization=best_configuration["reg"], batch_norm=best_configuration["bn"])
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        UNet_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        UNet_results = UNet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=best_configuration["epoch_num"], verbose=0)
        val_loss, _ = UNet_model.evaluate(x_val, y_val, verbose=0)
        print(f"val_loss={val_loss} (dr={dr})")
        print(f"val_loss={val_loss} (dr={dr})\n", file=f)

        if val_loss < best_configuration["val_loss"]:
            print("*** best_configuration updated")
            print("*** best_configuration updated\n", file=f)
            best_configuration["val_loss"] = val_loss
            best_configuration["loss"] = UNet_results.history["loss"]
            best_configuration["dr"] = dr

            UNet_model.save("UNet_model.h5")

    print(f"Test dropout rate - best configuration: {best_configuration}\n", file=f)

    for us in up_sample:
        UNet_model = Unet_Model(input_shape=(128,128,1), init_filter=16, drop_rate=best_configuration["dr"], up_sampling=us, 
                                regularization=best_configuration["reg"], batch_norm=best_configuration["bn"])
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        UNet_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        UNet_results = UNet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=best_configuration["epoch_num"], verbose=0)
        val_loss, _ = UNet_model.evaluate(x_val, y_val, verbose=0)
        print(f"val_loss={val_loss} (us={us})")
        print(f"val_loss={val_loss} (us={us})\n", file=f)

        if val_loss < best_configuration["val_loss"]:
            print("*** best_configuration updated")
            print("*** best_configuration updated\n", file=f)
            best_configuration["val_loss"] = val_loss
            best_configuration["loss"] = UNet_results.history["loss"]
            best_configuration["us"] = us

            UNet_model.save("UNet_model.h5")

    print(f"Test upsampling - best configuration: {best_configuration}\n", file=f)

    for bn in batch_norm:
        UNet_model = Unet_Model(input_shape=(128,128,1), init_filter=16, drop_rate=best_configuration["dr"], up_sampling=best_configuration["us"], 
                                regularization=best_configuration["reg"], batch_norm=bn)
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        UNet_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        UNet_results = UNet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=best_configuration["epoch_num"], verbose=0)
        val_loss, _ = UNet_model.evaluate(x_val, y_val, verbose=0)
        print(f"val_loss={val_loss} (bn={bn})")
        print(f"val_loss={val_loss} (bn={bn})\n", file=f)

        if val_loss < best_configuration["val_loss"]:
            print("*** best_configuration updated")
            best_configuration["val_loss"] = val_loss
            best_configuration["loss"] = UNet_results.history["loss"]
            best_configuration["bn"] = bn

            UNet_model.save("UNet_model.h5")

    print(f"Test batch normalization - best configuration: {best_configuration}\n", file=f)
f.close()

I0000 00:00:1730085053.753911   48097 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730085053.775704   48097 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730085053.775743   48097 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730085053.777705   48097 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730085053.777747   48097 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

val_loss=0.14003829658031464 (reg=None)
*** best_configuration updated
val_loss=0.768832266330719 (reg=l1)
val_loss=0.28613072633743286 (reg=l2)


val_loss=0.13951614499092102 (dr=0.1)
*** best_configuration updated


val_loss=0.13813170790672302 (dr=0.2)
*** best_configuration updated
val_loss=0.15285170078277588 (dr=0.3)
val_loss=0.16078081727027893 (dr=0.4)
val_loss=0.14754758775234222 (dr=0.5)
val_loss=0.15158602595329285 (us=False)
val_loss=0.13946713507175446 (us=True)
val_loss=0.182280033826828 (bn=True)


val_loss=0.13420577347278595 (bn=False)
*** best_configuration updated


In [6]:
with open("unet_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)
    
    # Test learning rate
    for lr in learning_rate:
        UNet_model = Unet_Model(input_shape=(128,128,1), init_filter=16, drop_rate=best_configuration["dr"], up_sampling=best_configuration["us"], 
                                regularization=best_configuration["reg"], batch_norm=best_configuration["bn"])
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        UNet_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        UNet_results = UNet_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=best_configuration["epoch_num"], verbose=0)
        val_loss, _ = UNet_model.evaluate(x_val, y_val, verbose=0)
        print(f"val_loss={val_loss} (lr={lr})")
        print(f"val_loss={val_loss} (lr={lr})\n", file=f)

        if val_loss < best_configuration["val_loss"]:
            print("*** best_configuration updated")
            print("*** best_configuration updated\n", file=f)
            best_configuration["val_loss"] = val_loss
            best_configuration["loss"] = UNet_results.history["loss"]
            best_configuration["lr"] = lr

            UNet_model.save("UNet_model.h5")

    print(f"Test learning rate - best configuration: {best_configuration}\n", file=f)
f.close()

I0000 00:00:1730087752.907516   73201 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730087753.211656   73201 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730087753.211737   73201 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730087753.214367   73201 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730087753.214400   73201 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

val_loss=0.19707901775836945 (lr=0.0001)
*** best_configuration updated


val_loss=0.14779502153396606 (lr=0.001)
*** best_configuration updated
val_loss=1.1295499801635742 (lr=0.01)
val_loss=1.1295499801635742 (lr=0.1)


# Tuned CNN

In [4]:
def encode(x):
    x[x=="normal"] = 0
    x[x=="benign"] = 1
    x[x=="malignant"] = 2
    x = x.astype(int)

    return x

In [5]:
def onehot_encode(x):
    x = encode(x)
    x_onehot = np.zeros((x.size, x.max()+1))
    x_onehot[np.arange(x.size),x] = 1

    return x_onehot

In [6]:
z_train_onehot = onehot_encode(z_train)
z_val_onehot = onehot_encode(z_val)
z_test_onehot = onehot_encode(z_test)

In [8]:
# Parameter Tuning

# Num Layer
number_layers = [3, 4, 5]

# Num Units
number_units = [16, 32, 64, 128, 256]

# Dropout Rates
drop_rates = [0.05, 0.1, 0.2, 0.3]

# Learning Rates
learning_rates = [0.0001, 0.001, 0.01, 0.1]

best_configuration = dict(
    val_acc=float(0), 
    num_layers=number_layers[0], 
    num_units=number_units[2], 
    dr=drop_rates[0], 
    lr=learning_rates[2], 
    epoch_num=10
)

In [9]:
with open("cnn_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)

    UNet_model = tf.keras.models.load_model("models/UNet_model.h5")

    for number_layer in number_layers:
        CNN_model = CNN_Model(input_shape=(128,128,1), num_layer=number_layer, num_unit=best_configuration["num_units"], drop_rate=best_configuration["dr"])
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        CNN_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        CNN_results = CNN_model.fit([x_train, UNet_model.predict(x_train)], z_train_onehot, validation_data=([x_val, UNet_model.predict(x_val)], z_val_onehot), epochs=best_configuration["epoch_num"], verbose=0)
        _, val_acc = CNN_model.evaluate([x_val, UNet_model.predict(x_val)], z_val_onehot, verbose=0)
        print(f"val_acc={val_acc} (num_layers={number_layer})")

        if val_acc > best_configuration["val_acc"]:
            print("*** best_configuration updated")
            best_configuration["val_acc"] = val_acc
            best_configuration["loss"] = CNN_results.history["loss"]
            best_configuration["val_loss"] = CNN_results.history["val_loss"]
            best_configuration["num_layers"] = number_layer

            CNN_model.save("CNN_model.h5")

    print(f"Test number of layers - best configuration: {best_configuration}\n", file=f)
f.close()

I0000 00:00:1730099686.144836   99549 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730099686.452155   99549 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730099686.452196   99549 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730099686.454788   99549 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730099686.454827   99549 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

10/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

I0000 00:00:1730099694.983874   99655 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


val_acc=0.7820512652397156 (num_layers=3)
*** best_configuration updated
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
val_acc=0.7051281929016113 (num_layers=4)
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
val_acc=0.7820512652397156 (num_layers=5)


In [ ]:
with open("cnn_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)

    UNet_model = tf.keras.models.load_model("models/UNet_model.h5")
    # Test number of units
    for number_unit in number_units:
        CNN_model = CNN_Model(input_shape=(128,128,1), num_layer=best_configuration["num_layers"], num_unit=number_unit, drop_rate=best_configuration["dr"])
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        CNN_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        CNN_results = CNN_model.fit([x_train, UNet_model.predict(x_train)], z_train_onehot, validation_data=([x_val, UNet_model.predict(x_val)], z_val_onehot), epochs=best_configuration["epoch_num"], verbose=0)
        _, val_acc = CNN_model.evaluate([x_val, UNet_model.predict(x_val)], z_val_onehot, verbose=0)
        print(f"val_acc={val_acc} (num_units={number_unit})")

        if val_acc > best_configuration["val_acc"]:
            print("*** best_configuration updated")
            best_configuration["val_acc"] = val_acc
            best_configuration["loss"] = CNN_results.history["loss"]
            best_configuration["val_loss"] = CNN_results.history["val_loss"]
            best_configuration["num_units"] = number_unit

            CNN_model.save("CNN_model.h5")

    print(f"Test number of units - best configuration: {best_configuration}\n", file=f)
f.close()

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


In [9]:
with open("cnn_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)

    UNet_model = tf.keras.models.load_model("models/UNet_model.h5")

    # Test dropout rate
    for dr in drop_rates:
        CNN_model = CNN_Model(input_shape=(128,128,1), num_layer=best_configuration["num_layers"], num_unit=best_configuration["num_units"], drop_rate=dr)
        optimizer = keras.optimizers.Adam(learning_rate=best_configuration["lr"])
        CNN_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        CNN_results = CNN_model.fit([x_train, UNet_model.predict(x_train)], z_train_onehot, validation_data=([x_val, UNet_model.predict(x_val)], z_val_onehot), epochs=best_configuration["epoch_num"], verbose=0)
        _, val_acc = CNN_model.evaluate([x_val, UNet_model.predict(x_val)], z_val_onehot, verbose=0)
        print(f"val_acc={val_acc} (dr={dr})")

        if val_acc > best_configuration["val_acc"]:
            print("*** best_configuration updated")
            best_configuration["val_acc"] = val_acc
            best_configuration["loss"] = CNN_results.history["loss"]
            best_configuration["val_loss"] = CNN_results.history["val_loss"]
            best_configuration["dr"] = dr

            CNN_model.save("CNN_model.h5")

    print(f"Test dropout rate - best configuration: {best_configuration}\n", file=f)

f.close()

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


: 

: 

In [ ]:
with open("cnn_tuning_output.txt", "a") as f:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Run at {current_time}\n", file=f)

    UNet_model = tf.keras.models.load_model("models/UNet_model.h5")

    # Test learning rate
    for lr in learning_rates:
        CNN_model = CNN_Model(input_shape=(128,128,1), num_layer=best_configuration["num_layers"], num_unit=best_configuration["num_units"], drop_rate=best_configuration["dr"])
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        CNN_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        CNN_results = CNN_model.fit([x_train, UNet_model.predict(x_train)], z_train_onehot, validation_data=([x_val, UNet_model.predict(x_val)], z_val_onehot), epochs=best_configuration["epoch_num"], verbose=0)
        _, val_acc = CNN_model.evaluate([x_val, UNet_model.predict(x_val)], z_val_onehot, verbose=0)
        print(f"val_acc={val_acc} (lr={lr})")

        if val_acc > best_configuration["val_acc"]:
            print("*** best_configuration updated")
            best_configuration["val_acc"] = val_acc
            best_configuration["loss"] = CNN_results.history["loss"]
            best_configuration["val_loss"] = CNN_results.history["val_loss"]
            best_configuration["lr"] = lr

            CNN_model.save("CNN_model.h5")

    print(f"Test learning rate - best configuration: {best_configuration}\n", file=f)
f.close()